In [ ]:
#imports
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np

## Load Data

In [ ]:
df = pd.read_csv('historica Iberdrola DIC 16-19.csv', delimiter=';')
df

,Fecha,Último
0,29-nov-2019,"8,928"
1,28-nov-2019,"8,916"
2,27-nov-2019,"8,928"
3,26-nov-2019,"8,896"
4,25-nov-2019,"8,898"
...,...,...
735,07-dic-2016,"5,729"
736,06-dic-2016,"5,775"
737,05-dic-2016,"5,581"
738,02-dic-2016,"5,599"


In [ ]:
import keras.backend as K
def porcentaje_margen(y_true, y_pred):
    margen = 0.02
    yy = K.sum(K.cast(K.less(K.abs((y_pred/y_true)-1.0), margen), dtype=float))
    return yy/K.cast(K.shape(y_pred)[0], dtype=float)

## Creating the Model

### Primera red

In [ ]:
model = keras.Sequential([
    keras.layers.SimpleRNN(10, activation='relu', return_sequences=True, input_shape=(20, 1)),
    keras.layers.SimpleRNN(5, activation='relu', return_sequences=False),
    keras.layers.Dense(1)
])

model.compile(
    optimizer = 'adam',
    loss = 'mse',
    metrics = [porcentaje_margen]
)

### Segunda Red - LSTM

In [ ]:
lstm = keras.Sequential([
    keras.layers.LSTM(5, activation="relu")
])

lstm.complile(
    optimizer = 'adam',
    loss = 'mse',
    metrics = [porcentaje_margen]
)

AttributeError: ignored

### Tercera red - GRU

In [ ]:
gru = keras.Sequential([
    keras.layers.GRU(5, activation="relu")
])

gru.complile(
    optimizer = 'adam',
    loss = 'mse',
    metrics = [porcentaje_margen]
)

AttributeError: ignored

## Training model

In [ ]:
# Shuffle data
from sklearn.utils import shuffle

In [ ]:
datosShuffled = shuffle(df, random_state=2)
datosShuffled

,Fecha,Último
715,05-ene-2017,"6,019"
640,24-abr-2017,"6,718"
142,13-may-2019,"8,062"
37,09-oct-2019,"9,352"
544,06-sep-2017,"6,85"
...,...,...
534,20-sep-2017,"6,713"
584,12-jul-2017,"6,875"
493,16-nov-2017,"6,544"
527,29-sep-2017,"6,572"


In [ ]:
# Division of a global sequences into n_step size ones (X) with the corresponding prediction desired (y)
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
X, y = split_sequence(df.Último, 20)

In [ ]:
X.shape, y.shape

((720, 20), (720,))

In [ ]:
X = X.reshape(720,20,1)
X.shape

(720, 20, 1)

In [ ]:
#y = df[20:] #a partir del 20 las salidas deseadas
#y

,Fecha,Último
20,01-nov-2019,"9,176"
21,31-oct-2019,"9,208"
22,30-oct-2019,"9,128"
23,29-oct-2019,"9,15"
24,28-oct-2019,"9,182"
...,...,...
735,07-dic-2016,"5,729"
736,06-dic-2016,"5,775"
737,05-dic-2016,"5,581"
738,02-dic-2016,"5,599"


In [ ]:
permu = shuffle(np.arange(720))
permu

array([434, 313, 594, 700, 204, 437, 236, 119, 454,  98, 468, 657, 498,
       134, 210, 555, 101, 695,  26,  17, 424,  45, 135, 588, 675, 353,
       341, 527,  52,   9, 279,  29, 453, 548,  38, 329,  51, 655, 324,
       714, 557, 249, 501, 365, 232, 392, 488, 458, 443,  40, 336, 515,
        99, 393, 600, 254, 428,  25, 253, 381, 536,  50, 409,  36, 308,
       327, 294,  66, 161, 201, 712, 300, 244, 223, 711, 352, 568, 624,
       716, 268,  74, 214, 323, 526, 371, 529, 203,   0, 678, 143, 480,
       233, 261, 200, 221, 252, 283, 598, 575, 169, 192, 547, 269,  73,
        82, 155, 614, 152, 349, 576, 332, 310, 257,  62, 358, 664, 649,
       307, 185, 246, 485, 263, 326, 319, 690, 694,  86, 187, 450, 622,
       670,  53, 126,  48,   2, 318, 653, 648, 719, 274, 590,  71, 356,
       592, 235,  68, 346,  13, 560, 290, 411, 288, 141, 627, 281, 230,
       602, 395, 558, 225, 601, 446, 640, 400, 642,  91, 525, 206,  65,
       234, 133, 451, 181,  95, 116, 570, 383, 360, 334, 523, 47

In [ ]:
idx_train = permu[:480]
idx_train

array([434, 313, 594, 700, 204, 437, 236, 119, 454,  98, 468, 657, 498,
       134, 210, 555, 101, 695,  26,  17, 424,  45, 135, 588, 675, 353,
       341, 527,  52,   9, 279,  29, 453, 548,  38, 329,  51, 655, 324,
       714, 557, 249, 501, 365, 232, 392, 488, 458, 443,  40, 336, 515,
        99, 393, 600, 254, 428,  25, 253, 381, 536,  50, 409,  36, 308,
       327, 294,  66, 161, 201, 712, 300, 244, 223, 711, 352, 568, 624,
       716, 268,  74, 214, 323, 526, 371, 529, 203,   0, 678, 143, 480,
       233, 261, 200, 221, 252, 283, 598, 575, 169, 192, 547, 269,  73,
        82, 155, 614, 152, 349, 576, 332, 310, 257,  62, 358, 664, 649,
       307, 185, 246, 485, 263, 326, 319, 690, 694,  86, 187, 450, 622,
       670,  53, 126,  48,   2, 318, 653, 648, 719, 274, 590,  71, 356,
       592, 235,  68, 346,  13, 560, 290, 411, 288, 141, 627, 281, 230,
       602, 395, 558, 225, 601, 446, 640, 400, 642,  91, 525, 206,  65,
       234, 133, 451, 181,  95, 116, 570, 383, 360, 334, 523, 47

In [ ]:
idx_test = permu[480:]
idx_test

array([635, 487, 415, 168,  96, 403, 610, 177, 347, 496, 691,  76, 130,
       184, 478, 198,  70, 581, 306, 265, 430, 542, 704, 277, 579, 282,
       227, 229, 245, 302, 587, 699, 465, 520, 384, 586,  20, 150,  49,
       120, 439, 684, 342, 363, 442, 406, 613, 284, 148, 368, 623, 164,
       607, 561, 113, 486, 145, 663, 619, 659,   6, 636,  81,  44, 473,
       686, 167, 276, 643, 414, 438, 471, 698, 136, 351, 413, 580,  43,
       172, 616, 611, 405, 621, 278, 545, 658, 577, 466, 387, 626,  90,
       373, 512,  92, 250, 435, 492, 299, 183, 718, 639,  56, 573, 154,
       121,  27, 491,  54, 255,  57, 401, 207, 146, 166, 505, 132, 516,
       502, 436, 503, 531, 661, 128, 222, 159, 112, 688,  37, 379, 398,
       652, 440, 211, 107,   1, 338, 706, 389,  41, 559, 556, 391, 219,
       646,  19, 189, 304, 412, 654, 584, 597, 108, 388, 386, 693, 474,
       682, 532,  39, 100, 321, 631, 197,  60, 110, 325, 140, 470, 293,
         5, 481,  18, 585, 228, 528, 603, 205, 417, 173, 188, 50

In [ ]:
X_train = X[idx_train]
X_train; X_train.shape

(480, 20, 1)

In [ ]:
X_test = X[idx_test]
X_test; X_test.shape

(240, 20, 1)

In [ ]:
Y_train = y[idx_train]
Y_train.shape

(480,)

In [ ]:
Y_test = y[idx_test]
Y_test.shape

(240,)

In [ ]:
history = model.fit(X_train, Y_train, epochs=100, validation_data=(X_test,Y_test),shuffle=False)

Epoch 1/100


UnimplementedError: ignored